In [1]:
import os
import re
import torch
from torch import nn
from tqdm import tqdm
from transformers import (
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    DataCollatorWithPadding,
    BitsAndBytesConfig,
)
from tokenizers.processors import TemplateProcessing
from trl import SFTConfig, SFTTrainer
from peft import (
    prepare_model_for_kbit_training, 
    LoraConfig, 
    get_peft_model,
    TaskType,
)
from datasets import load_dataset
from accelerate import Accelerator

/opt/conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 데이터셋 정의

In [2]:
repo = 'uomnf97/klue-roberta-finetuned-korquad-v2'
tokenizer = AutoTokenizer.from_pretrained(repo)
dataset = load_dataset("csv", data_files="/home/jovyan/work/prj_data/open/train.csv")
max_length = 512
stride = 160

def preprocess_function(examples):
    questions, contexts, answers = examples["question"], examples["context"], examples["answer"]
    def preprocess_text(text):
        text = text.replace('\n', ' ')
        text = re.sub(r'\s+', ' ', text).strip()
        return text
    questions = list(map(preprocess_text, questions))
    contexts = list(map(preprocess_text, contexts))
    answers = list(map(preprocess_text, answers))
    
    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # offset_mapping: [(token1 start, token1 end), (token2 ~, ), ...]
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs["overflow_to_sample_mapping"]# inputs.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    
    for i, offset in enumerate(offset_mapping):
        # sequence_ids: (token=None, question=0, context=1)
        sequence_ids = inputs.sequence_ids(i)

        # 컨텍스트의 시작 및 마지막을 찾는다.
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1
        
        sample_idx = sample_map[i]
        answer = answers[sample_idx]
        
        start_char = contexts[sample_idx].find(answer, offset[context_start][0], offset[context_end][1])
        end_char = start_char + len(answer)

        # 만일 정답이 컨텍스트에 완전히 포함되지 않는다면, 레이블은 (0, 0)임
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # 그렇지 않으면 정답의 시작 및 마지막 인덱스
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)
    
            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# 데이터 프레임을 전처리합니다
# preprocess_function(dataset["train"][15281])
train_dataset = dataset["train"].map(
    preprocess_function,
    batched = True,
    remove_columns=dataset["train"].column_names,
)

# roBERTa에서는 삭제, BERT에서는 중요한 역할
train_dataset = train_dataset.remove_columns("token_type_ids")

Map: 100%|██████████| 33716/33716 [00:52<00:00, 642.43 examples/s]


In [3]:
idx = 9
sample_idx = train_dataset["overflow_to_sample_mapping"][idx]
answer = dataset['train']['answer'][sample_idx]

start = train_dataset['start_positions'][idx]
end = train_dataset['end_positions'][idx]
labeled_answer = tokenizer.decode(train_dataset["input_ids"][idx][start : end + 1])

print(f"Theoretical answer: {answer}, labels give: {labeled_answer}")
print(tokenizer.decode(train_dataset["input_ids"][idx]))

Theoretical answer: 6세대, labels give: 6세대
[CLS] PM9A3 E1. S가 기반한 V낸드는 몇 세대인가 [SEP] 데이터 암호화와 같은 기본적인 보안 기능 뿐만 아니라 안티롤백, 보안 부팅 등 다양한 보안 솔루션을 제공한다. 안티롤백은 보안이 취약한 하위 버전의 펌웨어가 다운로드 되지 못하도록 막는 기능으로, PM9A3 E1. S는 보안 취약점이 발견된 펌웨어에 대해서는 이력을 따로 저장해놓고 해당 버전을 다운로드할 경우 정상적으로 처리되지 않도록 막는다. 이번 SSD는 기본적으로 허가되지 않은 접근이 안되도록 설계돼 있지만, 보안 부팅 기능을 추가해 보안을 더욱 강화했다. 보안 부팅은 SSD 내부에 갖고 있는 전자서명을 부팅 과정에서 체크해, 정상적으로 인식되는 경우에만 부팅이 될 수 있도록 한다. 삼성전자 메모리사업부 상품기획팀 박철민 상무는 " PM9A3 E1. S는 6세대 V낸드 기반으로 업계 최고 수준의 전력 효율을 구현한 NVMe SSD로 대규모 데이터센터 고객들에게 최적의 솔루션이 될 것 " 이라며, " 향후 OCP에 참여한 다양한 고객사들과 협력해 데이터센터용 SSD 표준을 만들어 나갈 것 " 이라고 밝혔다. 페이스북의 OCP SSD 총괄 로스 스텐포트는 “ OCP NVMe 클라우드 SSD는 최근 데이터센터의 기술적 난제를 해결하는데 매우 중요한 실마리를 제시해준다 ” 면서 “ 특히 삼성전자의 이번 제품은 대규모 확장이 필요한 데이터센터 환경에 적합한 SSD 요구 사양을 충족하고 있다 ” 고 밝혔다. 삼성전자는 이번 PM9A3 E1. S 양산을 시작으로 5G 시대의 본격 개막과 초연결 라이프 스타일로 변화하는 시대에 발맞춰 글로벌 데이터센터 업체들과 지속 협력하며 차세대 기술 확보와 표준화를 주도해 나갈 계획이다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD

# 모델 정의

In [3]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
print("start")
model = AutoModelForQuestionAnswering.from_pretrained(
        repo,
        device_map="cuda:0",
        torch_dtype=torch.float32,
        quantization_config=quantization_config,
)
print("end")
model.resize_token_embeddings(len(tokenizer))

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    target_modules=["query", "key", "value"],
    task_type="QUESTION_ANSWERING"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

accelerater = Accelerator()
model, tokenizer = accelerater.prepare(model, tokenizer)

start


KeyboardInterrupt: 

# Loss 정의

# 학습

In [ ]:
import wandb
wandb.login()

torch.cuda.empty_cache()
training_args = TrainingArguments(
    output_dir="RoBERTa_v2",
    evaluation_strategy="no",
    num_train_epochs=4,
    save_steps=0.1,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=1,
    max_grad_norm=1.0,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10
)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer)
)

In [5]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


/opt/conda/envs/qa/lib/python3.9/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
10,2.642200
20,2.753300
30,1.907300
40,2.137500
50,1.002200


KeyboardInterrupt: 

# Inference

In [1]:
import torch
import pandas as pd
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from tqdm import tqdm

CHECK_POINT = "/home/jovyan/work/ai_chat_qa_task/code/huggingface/roBERTa_v1_answer/checkpoint-24168"
TEST_fOLDER = '/home/jovyan/work/prj_data/open/test.csv'
OUTPUT = "roberta_v1_answer"

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
csv = pd.read_csv(TEST_fOLDER)

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

# 모델 및 토크나이저 로드
config = PeftConfig.from_pretrained(CHECK_POINT)
model = AutoModelForQuestionAnswering.from_pretrained(
    config.base_model_name_or_path,
    # quantization_config=quantization_config,
    device_map="cuda:0",
    torch_dtype=torch.float16
)
# model = PeftModel.from_pretrained(model, CHECK_POINT)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.add_tokens(['왓챠', '대만 서부', '먀오리현', '爭', '訟'])
model.resize_token_embeddings(len(tokenizer))

/opt/conda/envs/qa/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Embedding(32005, 1024, padding_idx=1)

In [2]:
import re
TEST_fOLDER = '/home/jovyan/work/prj_data/open/test.csv'
csv = pd.read_csv(TEST_fOLDER)

tokenizer.add_tokens(['왓챠', '今', '昨', '苗', '栗', '縣', '寒', '진퀀텀', '±', '･', '훙멍'])
model.resize_token_embeddings(len(tokenizer))

def get_prediction(question, context):
    inputs = tokenizer(
        question,
        context,
        max_length=512,
        return_tensors="pt",
        truncation="only_second",
        stride=256,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    inputs.pop("token_type_ids")
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    
    inputs = {k: v.to(DEVICE) for k, v in inputs.items()}

    
    with torch.no_grad():
        outputs = model(**inputs)

    # 모든 청크에 대한 start/end 로짓 가져오기
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    
    # 가장 높은 점수의 답변 찾기
    max_answer_score = -float('inf')
    best_answer = ""

    for i in range(len(start_logits)):
        start_indexes = torch.argsort(start_logits[i], descending=True)[:20]
        end_indexes = torch.argsort(end_logits[i], descending=True)[:20]
        
        for start_index in start_indexes:
            for end_index in end_indexes:
                # 답변의 길이를 50 토큰으로 제한
                if end_index < start_index or end_index - start_index + 1 > 50 or end_index - start_index <= 1:
                    continue
                # 답변이 CLS일때는 제외
                if start_index==0 or end_index==0:
                    continue
                    
                answer_score = start_logits[i][start_index] + end_logits[i][end_index]
                if answer_score > max_answer_score:
                    decoded_answer = tokenizer.decode(inputs["input_ids"][i][start_index:end_index+1])
                    # 답변에 SEP가 포함되면 제외
                    if '[SEP]' not in decoded_answer:
                        max_answer_score = answer_score
                        best_answer = decoded_answer
    return best_answer
    
submission_dict = {}
i = -1
for _, row in tqdm(csv.iterrows(), total=len(csv)):
    i += 1
    answer = get_prediction(row['question'], row['context'])

    def clean_prediction(text):
        special_tokens = list(tokenizer.special_tokens_map.values())
        pattern = '|'.join(map(re.escape, special_tokens))
        cleaned_text = re.sub(pattern, '', text)
        cleaned_text = ' '.join(cleaned_text.split())
        cleaned_text = re.sub(r'##?', '', text)
        return cleaned_text

    answer = clean_prediction(answer)
    submission_dict[row['id']] = answer
    # print(f"ID: {row['id']}\n Question: {row['question']}\n Generated answer: {answer}")
df = pd.DataFrame(list(submission_dict.items()), columns=['id', 'answer'])

100%|██████████| 1507/1507 [36:12<00:00,  1.44s/it] 


In [3]:
df.to_csv('robert_stride256.csv', index=False)

In [ ]:
497 795 866 986 1203 1326